In [1]:
#!unzip Test.zip

In [2]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
suffix = '.pgm'
kaggle_files = os.listdir('Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [9]:
len(pgm_kaggle_files)

8000

In [10]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = 'Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

  0%|          | 0/8000 [00:00<?, ?it/s]

100%|██████████| 8000/8000 [00:04<00:00, 1923.46it/s]


In [11]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 1068510.40it/s]


In [12]:
import joblib

In [21]:
# Aquí debe utilizar su modelo
model = joblib.load('models/best_knn_model.joblib')

In [22]:
from skimage import feature

# Aquí debe procesar las imagenes de pgm_kaggle_files
X_kag = []

suffix = '.pgm'
dir = os.listdir('Test/')
pgm_files = [filename for filename in dir if filename.endswith(suffix)]

for filename in tqdm(pgm_files):
    path = 'Test/' + filename
    with open (path, 'rb') as pgmf:
        image = plt.imread(pgmf)
        X_kag.append(image)
        
X = np.array([feature.hog(im) for im in tqdm(X_kag, desc='Construyendo X') ])


Construyendo X: 100%|██████████| 8000/8000 [00:10<00:00, 775.41it/s]


In [15]:
pca_model = joblib.load('models/pca_model.joblib')

In [23]:
X_pca_kag = pca_model.fit_transform(X)

# Predicción
y_kag = model.predict(X_pca_kag)

: 

In [17]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [18]:
kaggle_hat.head()

,id,target_feature
2069,0,1
2527,1,0
3030,2,1
2576,3,1
4060,4,1


In [19]:
from datetime import datetime

fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M")

print(fecha_hora)

2025-06-11 23:37


In [20]:
# Guardamos en un .csv para subir a kaggle
submission_name = f'knn_{fecha_hora}'
kaggle_hat.to_csv(submission_name + '.csv', index=False)